In [ ]:
pip install keras_preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import os
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Dropout ,Flatten , MaxPooling2D

In [ ]:
!pip install kaggle

In [ ]:
# API to fetch the dataset from Kaggle
!kaggle datasets download -d msambare/fer2013

Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
 78% 47.0M/60.3M [00:00<00:00, 91.2MB/s]
100% 60.3M/60.3M [00:00<00:00, 113MB/s] 


In [ ]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/fer2013.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
train_dir='/content/train'
test_dir='/content/test'

In [ ]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,'completed')
    return image_paths,labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(train_dir)


surprise completed
sad completed
fear completed
neutral completed
angry completed
happy completed
disgust completed


In [ ]:
print(train)

                                               image     label
0      /content/train/surprise/Training_32304486.jpg  surprise
1      /content/train/surprise/Training_59119116.jpg  surprise
2      /content/train/surprise/Training_66774841.jpg  surprise
3      /content/train/surprise/Training_39862056.jpg  surprise
4      /content/train/surprise/Training_58363387.jpg  surprise
...                                              ...       ...
28704   /content/train/disgust/Training_94788538.jpg   disgust
28705    /content/train/disgust/Training_5242503.jpg   disgust
28706   /content/train/disgust/Training_14124662.jpg   disgust
28707   /content/train/disgust/Training_71118638.jpg   disgust
28708   /content/train/disgust/Training_97638867.jpg   disgust

[28709 rows x 2 columns]


In [ ]:
test=pd.DataFrame()
test['image'],test['label']=createdataframe(test_dir)

surprise completed
sad completed
fear completed
neutral completed
angry completed
happy completed
disgust completed


In [ ]:
print(test)

                                                image     label
0      /content/test/surprise/PublicTest_55075980.jpg  surprise
1     /content/test/surprise/PrivateTest_46434045.jpg  surprise
2     /content/test/surprise/PrivateTest_66702303.jpg  surprise
3     /content/test/surprise/PrivateTest_25532192.jpg  surprise
4     /content/test/surprise/PrivateTest_49077480.jpg  surprise
...                                               ...       ...
7173   /content/test/disgust/PrivateTest_46114477.jpg   disgust
7174   /content/test/disgust/PrivateTest_36627671.jpg   disgust
7175   /content/test/disgust/PrivateTest_38750068.jpg   disgust
7176    /content/test/disgust/PublicTest_48953105.jpg   disgust
7177   /content/test/disgust/PrivateTest_91247555.jpg   disgust

[7178 rows x 2 columns]


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


In [ ]:
train_features = extract_features(train['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
test_features = extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Initialize the model
model = Sequential()

# Convolutional layers with Batch Normalization
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate reduction
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Train the model
# model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64, callbacks=[lr_scheduler])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, batch_size=64, callbacks=[lr_scheduler])


Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 1688s 4s/step - accuracy: 0.2071 - loss: 2.2352 - val_accuracy: 0.2892 - val_loss: 1.9820 - learning_rate: 0.0010
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 1670s 4s/step - accuracy: 0.3645 - loss: 1.6298 - val_accuracy: 0.3958 - val_loss: 1.5325 - learning_rate: 0.0010
Epoch 3/50
 59/449 ━━━━━━━━━━━━━━━━━━━━ 23:05 4s/step - accuracy: 0.4597 - loss: 1.3720

In [ ]:
import numpy as np
import cv2  # For image processing
from keras.models import load_model
import matplotlib.pyplot as plt

# Load the trained model


# Define emotion labels (adjust based on your dataset)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Load and preprocess the test image
def preprocess_image(img_path):
    # Load image in grayscale mode
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Resize to match the input shape of the model
    img = cv2.resize(img, (48, 48))

    # Normalize pixel values to range [0, 1]
    img = img.astype('float32') / 255.0

    # Reshape to (1, 48, 48, 1) for prediction
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)

    return img

# Test image path
test_image_path = '/IMG20241118142550.jpg'   # Replace with the path to your test image

# Preprocess the image
preprocessed_image = preprocess_image(test_image_path)

# Predict the emotion
prediction = model.predict(preprocessed_image)
predicted_emotion = emotion_labels[np.argmax(prediction)]

# Display the test image and prediction
plt.imshow(cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE), cmap='gray')
plt.title(f'Predicted Emotion: {predicted_emotion}')
plt.axis('off')
plt.show()

print(f'Predicted Emotion: {predicted_emotion}')


In [ ]:
def detect_and_predict_emotions(img_path):
    # Read the image
    img = cv2.imread(img_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        print("No face detected.")
        return

In [ ]:
x_test.shape

(7178, 48, 48, 1)

In [ ]:
y_test.shape

(7178, 7, 7)